**Install Required Dependencies**

In [ ]:
pip install -r requirements_quantum.txt

**Import Libraries**

In [ ]:
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
import folium
import itertools
from geopy.distance import geodesic
import time
from typing import List, Dict, Tuple
import matplotlib.pyplot as plt
import requests
import json
import math
from itertools import permutations

ox.settings.use_cache = True
ox.settings.log_console = True

**Define Distance Metric Function (OSRM)**

In [ ]:
# function to get distance and duration between two coordinates using OSRM
def metric(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    coords_str = f"{lon1},{lat1};{lon2},{lat2}"
    url = f"http://router.project-osrm.org/route/v1/driving/{coords_str}?overview=full&geometries=geojson"

    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()

        if data['code'] == 'Ok' and data['routes']:
            route = data['routes'][0]
            distance_km = route['distance'] / 1000  # meters → km
            duration_min = route['duration'] / 60   # seconds → minutes
            geometry = route['geometry']['coordinates']
            return distance_km
        else:
            print("OSRM error: No route found")
            return None
    except requests.exceptions.RequestException as e:
        print(f"OSRM API error: {e}")
        return None


**Load Data and Initialize Quantum Optimization Problem**

In [ ]:
# importing and get the data
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import StatevectorSampler
from qiskit_optimization.converters import QuadraticProgramToQubo
from IPython.display import display, Math

with open("OptimizationProblemData.json", "r") as f:
    data = json.load(f)

hospital = data["locations"]["hospital"]["coordinates"]
patients = data["locations"]["patients"]
n_patients = len(patients)
max_stops = 3

locations = [hospital] + [p["coordinates"] for p in patients]
n_locations = len(locations)

**Build Distance Matrix and Formulate Quadratic Program (Ambulance Routing)**

In [ ]:
# init the data
# distance matrix
dist_matrix = np.zeros((n_locations, n_locations))
for i in range(n_locations):
    for j in range(n_locations):
        if i != j:
            coord_i = locations[i]
            coord_j = locations[j]
            dist_matrix[i, j] = metric(
                coord_i["latitude"], coord_i["longitude"],
                coord_j["latitude"], coord_j["longitude"]
            )

# create Quadratic Program
# create the model
qp = QuadraticProgram(name="Ambulance_Routing")

# Binary variables x_{i,t}: patient i is visited in trip t
for i in range(n_patients):
    for t in range(2):  # at most 2 trips
        qp.binary_var(name=f"x_{i}_{t}")

# each patient is visited exactly once
for i in range(n_patients):
    constraint_terms = {f"x_{i}_{t}": 1 for t in range(2)}
    qp.linear_constraint(
        linear=constraint_terms, sense="==", rhs=1,
        name=f"patient_{i}_visited_once"
    )


# each trip can visit at most max_stops patients
for t in range(2):
    constraint_terms = {f"x_{i}_{t}": 1 for i in range(n_patients)}
    qp.linear_constraint(
        linear=constraint_terms, sense="<=", rhs=max_stops,
        name=f"trip_{t}_max_patients"
    )

# objective, minimize travel distance
linear_terms = {}
quadratic_terms = {}

for t in range(2):
    for i in range(n_patients):
        linear_terms[f"x_{i}_{t}"] = dist_matrix[0, i+1] * 0.5
    for i in range(n_patients):
        for j in range(i+1, n_patients):
            quadratic_terms[(f"x_{i}_{t}", f"x_{j}_{t}")] = dist_matrix[i+1, j+1] * 0.3

**Solve Quadratic Program with QAOA and Validate Solutions**

In [ ]:
# try and solve

qp.minimize(linear=linear_terms, quadratic=quadratic_terms)

# to qubo
converter = QuadraticProgramToQubo()
qubo = converter.convert(qp)

# qaoa
optimizer = COBYLA(maxiter=50)
qaoa = QAOA(sampler=StatevectorSampler(), optimizer=optimizer, reps=10)
algorithm = MinimumEigenOptimizer(qaoa)

# check if qaoa is return a valid solution after del the third constraine
def is_valid_solution(trips: List[List[str]], max_stops: int, n_patients: int) -> bool:
    visited = [p for trip in trips for p in trip]
    if len(visited) != n_patients or len(set(visited)) != n_patients:
        return False
    for trip in trips:
        if len(trip) > max_stops:
            return False
    return True

# run qaoa until valid solution is found or max retries reached
def run_qaoa_until_valid(max_retries=20):
    for attempt in range(max_retries):
        print(f"\n--- Attempt {attempt+1} ---")
        result = algorithm.solve(qubo)

        trips = [[], []]
        for i in range(n_patients):
            for t in range(2):
                var_name = f"x_{i}_{t}"
                if var_name in result.variables_dict and result.variables_dict[var_name] > 0.5:
                    trips[t].append(patients[i]["id"])

        if is_valid_solution(trips, max_stops, n_patients):
            print("Found valid solution!")
            return trips, result

        print("Invalid solution, retrying...")

    raise ValueError("No valid solution found after max retries.")


# calculate total distance for a trip
def calculate_trip_distance(patient_ids):
    if not patient_ids:
        return 0, []

    patient_indices = []
    for pid in patient_ids:
        for i, p in enumerate(patients):
            if p["id"] == pid:
                patient_indices.append(i+1)

    min_distance = float('inf')
    best_order = []

    for order in permutations(patient_indices):
        distance = dist_matrix[0, order[0]]
        for j in range(len(order)-1):
            distance += dist_matrix[order[j], order[j+1]]
        distance += dist_matrix[order[-1], 0]

        if distance < min_distance:
            min_distance = distance
            best_order = [patients[idx-1]["id"] for idx in order]

    return min_distance, best_order


**Run QAOA Solver and Display Optimized Ambulance Trips**

In [ ]:

# main
valid_trips, result = run_qaoa_until_valid(max_retries=30)

total_distance = 0
for t, trip in enumerate(valid_trips):
    if trip:
        distance, order = calculate_trip_distance(trip)
        total_distance += distance
        print(f"Trip {t+1}: Hospital -> {' -> '.join(order)} -> Hospital (Distance: {distance:.2f} km)")
    else:
        print(f"Trip {t+1}: Empty")

print(f"\nTotal distance: {total_distance:.2f} km")

print("\nPatient coordinates:")
for i, patient in enumerate(patients):
    print(f"{patient['id']}: ({patient['coordinates']['latitude']:.6f}, {patient['coordinates']['longitude']:.6f})")
print(f"Hospital: ({hospital['latitude']:.6f}, {hospital['longitude']:.6f})")



--- Attempt 1 ---
Invalid solution, retrying...

--- Attempt 2 ---
Found valid solution!
Trip 1: Hospital -> R3_2 -> R2 -> Hospital (Distance: 26.79 km)
Trip 2: Hospital -> DT -> GR -> IT -> Hospital (Distance: 30.74 km)

Total distance: 57.52 km

Patient coordinates:
DT: (30.000418, 31.739608)
GR: (30.011344, 31.747827)
R2: (30.030388, 31.669231)
R3_2: (30.030941, 31.688371)
IT: (30.012856, 31.693812)
Hospital: (29.995127, 31.684628)
